# Pair Solution.md

## Decision Trees

Decision trees are a recursive divide and conquer algorithm. They are a non-linear, non-parametric discriminative supervised classification algorithm.  There are a few names of decision tree algorithms you may have heard of (ID3, C4.5, CART, etc.) and each is a different specification of a decision tree model.  You can read about them [here](http://stackoverflow.com/questions/9979461/different-decision-tree-algorithms-with-comparison-of-complexity-or-performance) and [here](http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart).

### Play Golf Dataset

When implementing any ML algorithm for the first time, it is often easier to start with a trivially simple data set. You should always focus on one portion of the pipeline at a time: we do not want worry about cleaning data during feature selection just as we do not want to worry about feature engineering when writing our model building code.  We will be using the canonical 'Play Golf' [dataset](http://www2.cs.uregina.ca/~dbd/cs831/notes/ml/dtrees/c4.5/c4.5_prob1.html) when writing our algorithm.

Look at the [golf data](data/playgolf.csv). You will also see a dataset with just the categorial features and one with just the continuous features. Starting with just categorical features may be easier for implementation.

### Pseudo-code

Here's the pseudocode for the algorithm you will be implementing.

    function BuildTree:
        If every item in the dataset is in the same class
        or there is no feature left to split the data:
            return a leaf node with the class label
        Else:
            find the best feature and value to split the data
            split the dataset
            create a node
            for each split
                call BuildTree and add the result as a child of the node
            return node

### Implementation

You've been given starter code in the [src](src) folder. Some of the instance variables chosen are not the only possible way of implementing a decision tree, so feel free to modify anything if it fits your implementation better.

* The `TreeNode` class is implemented. These are the instance variables:

    * `column` (int): index of feature to split on
    * `split_value` (object): value of the feature to split on
    * `categorical` (bool): whether or not node is split on a categorial feature (vs continuous)
    * `name` (string): name of the feature (or name of the class in the case of a list)
    * `left` (TreeNode): left child
    * `right` (Tree Node): right child
    * `leaf` (boolean): true or false depending on if the node is a leaf node.
    * `classes` (Counter): if a leaf, a count of all the list of all the classes of the data points that terminate at this leaf.  Can be used to assess how "accurate" an individual leaf is.

    The `as_string` and `__str__` functions are designed to print out the decision tree (mostly for debugging).

* There is starter code for the `DecisionTree` class. You will need to fill in the class so that you can use your decision tree code as follows (also see `src/run_decision_tree.py`).

    ```python
    tree = DecisionTree()
    tree.fit(X, y, df.columns[:-1])
    print tree
    y_predict = tree.predict(X)
    ```

    You can see that the `__str__` method is implemented for you. This enables you to print your tree for debugging purposes.

* The `__init__`, `fit`, `_build_tree` and `__str__` methods are already implemented for you. You will need to implement the other ones.

* There are minimal tests in `src/test_decision_tree.py`. One test for each method you need to implement. You can run the tests with this command:

    ```
    nosetests src/test_decision_tree.py
    ```

* The file `run_decision_tree.py` should run your Decision Tree code, print the resulting decision tree and show the predicted results.

### Steps to Implementing

We will be implementing the **CART** algorithm. This means that every split will be binary. For categorical features, splits will be like: `sunny` or `not sunny`. For continuous features, splits will be like: `<80` or `>=80`.

1. Implement the `_entropy` method, which is given by the following equation. Entropy measures the amount of "disorder" in a set. Here there are *m* classes in the set and *ci* is the *i*-th class of our target y.

    ![shannon entropy](images/entropy.png)

    *P(c)* = (count of occurrences of class *c*) / size of *y*

    Note that to calculate entropy, you only need the labels (`y` values) and none of the feature values.

2. Implement the `_gini` method. Your information gain method will be able to use either gini or entropy.

    ![gini impurity](images/gini.png)

3. Implement the `_make_split` method. This should take the index of the feature and the value of the feature and make the split of the data into two subsets. Note that for categorical features this should split on whether it's equal to the value or not. For continuous, it should split on `<` or `>=`.

4. Implement the `_information_gain` method. This should take a split (the result of the `_make_split` method) and return the value of the information gain.

5. Implement the `_choose_split_index` method. This should take the data and try every possible feature and value to split on. It should find the one with the best information gain.

6. The `predict` method in the Decision Tree class is implemented by calling the `predict_one` method in the `TreeNode` class. You need to finish the implementation of the `predict_one` method by giving the conditions for when you should move left or right on the decision tree.


### Extra Credit 1: Pruning

*Pruning* is designed to simplify the tree so it doesn't go so deep. It is a way of stopping earlier or merging leaves that helps deal with overfitting. The first two extra credit problems are implementing prepruning and postpruning. A well designed decision tree would have these implemented.

1. *Prepruning* is making the decision tree algorithm stop early. Here are a few ways that we preprune:
    * leaf size: Stop when the number of data points for a leaf gets below a threshold
    * depth: Stop when the depth of the tree (distance from root to leaf) reaches a threshold
    * mostly the same: Stop when some percent of the data points are the same (rather than all the same)
    * error threshold: Stop when the error reduction (information gain) isn't improved significantly.

    Implement some of the prepruning thresholds and play around with using them.

2. Implement *postpruning* for your decision tree. You build the tree the same as before, but after you've built the tree, merge some nodes together if doing so reduces the test-set error. Here's the psuedocode:

        function Prune:
            if either left or right is not a leaf:
                call Prune on that split
            if both left and right are leaf nodes:
                calculate error associated with merging two nodes
                calculate error associated without merging two nodes
                if merging results in lower error:
                    merge the leaf nodes

    You can find more detail in section 9.4.2 in Machine Learning in Action.


### Extra Credit 2: Decision Trees for Regression

**Note:** Before starting this, make sure you commit your code with a `git commit`! Don't lose your past results with your new changes!

You can use decision trees for predicting continuous values as well. Instead of using entropy to calculate the disorder in the set, we use the variance.

To get to value of a leaf node, average all of the values.

1. Make your decision tree able to predict continuous values. You can modify your decision tree class so that it can do either continuous or categorical depending on what parameters you pass it, or just copy and create a new class. For checking out if your code is implemented correctly, you can use the same dataset and predict one of the continuous variables.

2. Implement model trees, which are predictors which start by using a decision tree, but use linear regression to predict the value on each leaf node. Details can be found in 9.5 of Machine Learning in Action.


### Extra Credit 3: A Real Dataset

1. Try running your decision tree code on a previous exercise's dataset.

2. Use sklearn's [Decision Tree](http://scikit-learn.org/stable/modules/tree.html#classification) and [k Nearest Neighbors](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) classifiers on the same dataset. How well do they do compared to logistic regression?

3. Implement model trees, which are predictors which start by using a decision tree, but use linear regression to predict the value on each leaf node. Details can be found in 9.5 of Machine Learning in Action.

# Soln

In [12]:
# COLLAPSE CELL
from collections import Counter
from TreeNode import TreeNode
from DecisionTreePruning import DecisionTreePruning
from itertools import izip
from DecisionTreeRegressor import DecisionTreeRegressor
# PMsearch np.v*
#x = data['mass']
#x?

# from jupyterthemes import jtplot
# jtplot.style(theme='solarized')
# from jupyterlab_table import JSONTable
# JSONTable(df)

# from IPython.display import HTML, display

# from notebook.services.config import ConfigManager
# cm = ConfigManager().update('notebook', {'limit_output': 1000})

import better_exceptions
better_exceptions.MAX_LENGTH = None

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error

from pprint import pprint
import math
import statsmodels.stats as sms
import statsmodels.api as sm
import statsmodels.regression as smr
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

# 04atplotlib inline
# %load_ext heat

plt.ion()
# plt.ioff()

# %heat

import os 
# dir_path = os.path.dirname(os.path.realpath(__file__))
cwd = os.getcwd()

# fig, ax = plt.subplots()
ax.plot(x, y)

/Users/tomwong/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [13]:
# TreeNode.py
"""Contains a node class for a decision tree."""

# from collections import Counter
# import numpy as np


class TreeNode(object):
    """A node class for a decision tree."""

    def __init__(self):
        """Initialize an empty TreeNode."""
        self.column = None  # (int)    index of feature to split on
        self.value = None  # value of the feature to split on
        self.categorical = True  # (bool) whether or not node is split on
                                 # categorial feature
        self.name = None    # (string) name of feature (or name of class in the
                            #          case of a list)
        self.left = None    # (TreeNode) left child
        self.right = None   # (TreeNode) right child
        self.leaf = False   # (bool)   true if node is a leaf, false otherwise
        self.classes = Counter()  # (Counter) only necessary for leaf node:
                                  #           key is class name and value is
                                  #           count of the count of data points
                                  #           that terminate at this leaf

    def predict_one(self, x):
        """Return the predicted label for a single data point.

        Parameters
        ----------
        x: 1d numpy array
            A single data point.

        Returns
        -------
        y: predicted label
        """
        if self.leaf:
            return self.name
        col_value = x[self.column]

        if self.categorical:
            if col_value == self.value:
                return self.left.predict_one(x)
            else:
                return self.right.predict_one(x)
        else:
            if col_value < self.value:
                return self.left.predict_one(x)
            else:
                return self.right.predict_one(x)

    # This is for visualizing your tree. You don't need to look into this code.
    def as_string(self, level=0, prefix=""):
        """Return a string representation of the tree rooted at this node.

        Parameters
        ----------
        level: int
            Amount to indent.

        Returns
        -------
        prefix: str
            Prefix to start the line with.
        """
        result = ""
        if prefix:
            indent = "  |   " * (level - 1) + "  |-> "
            result += indent + prefix + "\n"
        indent = "  |   " * level
        result += indent + "  " + str(self.name) + "\n"
        if not self.leaf:
            if self.categorical:
                left_key = str(self.value)
                right_key = "no " + str(self.value)
            else:
                left_key = "< " + str(self.value)
                right_key = ">= " + str(self.value)
            result += self.left.as_string(level + 1, left_key + ":")
            result += self.right.as_string(level + 1, right_key + ":")
        return result

    def __repr__(self):
        """Represent TreeNode as string."""
        return self.as_string().strip()


In [14]:
# DecisionTree.py

"""
Class implementing the CART decision tree algorithm.

"""

# import pandas as pd
# import numpy as np
# import math
# from collections import Counter
# from TreeNode import TreeNode


class DecisionTree(object):
    """Classifier implementing the CART decision tree algorithm.

    Parameters
    ----------
    impurity_criterion: string, optional (default='entropy')
        String indicating the impurity_criterion to use.
        Use 'gini' to have tree us Gini impurity.
    """

    def __init__(self, impurity_criterion='entropy'):
        """Initialize an empty DecisionTree."""
        # Root Node
        self.root = None
        # String names of features (for interpreting the tree)
        self.feature_names = None
        # Boolean array of whether variable is categorical (or continuous)
        self.categorical = None

        if impurity_criterion == 'entropy':
            self.impurity_criterion = self._entropy
        else:
            self.impurity_criterion = self._gini


    def fit(self, X, y, feature_names=None):
        """Build the decision tree.

        Parameters
        ----------
        X: 2d numpy array, shape = [n_samples, n_features]
            The training data.
        y: 1d numpy array, shape = [n_samples]
            The training labels.
        feature_names: numpy array, optional (default=None)
            Array of strings containing names of each of the features.

        Returns
        -------
        None
        """
        if feature_names is None or len(feature_names) != X.shape[1]:
            self.feature_names = np.arange(X.shape[1])
        else:
            self.feature_names = feature_names

        # Create True/False array of whether the variable is categorical
        is_categorical = lambda x: isinstance(x, str) or isinstance(x, bool)
        self.categorical = np.vectorize(is_categorical)(X[0])

        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y):
        """Recursively build the decision tree.

        Return the root node.

        Parameters
        ----------
        X: 2d numpy array, shape = [n_samples, n_features]
            The training data.
        y: 1d numpy array, shape = [n_samples]

        Returns
        -------
        TreeNode
        """
        node = TreeNode()
        index, value, splits = self._choose_split_index(X, y)

        if index is None or len(np.unique(y)) == 1:
            node.leaf = True
            node.classes = Counter(y)
            node.name = node.classes.most_common(1)[0][0]
        else:
            X1, y1, X2, y2 = splits
            node.column = index
            node.name = self.feature_names[index]
            node.value = value
            node.categorical = self.categorical[index]
            node.left = self._build_tree(X1, y1)
            node.right = self._build_tree(X2, y2)
        return node

    def _entropy(self, y):
        """Return the entropy of the array y.

        Parameters
        ----------
        y: 1d numpy array
            An array of data.

        Returns
        -------
        float
            Entropy of the array y.
        """
        n = y.shape[0]
        summation = 0
        for c_i in np.unique(y):
            prob = np.mean(y == c_i)
            summation += prob * np.log2(prob)
        return -summation

    def _gini(self, y):
        """Return the gini impurity of the array y.

        Parameters
        ----------
        y: 1d numpy array
            An array of data

        Returns
        -------
        float
            Gini impurity of the array y.
        """
        n = y.shape[0]
        summation = 0
        for c_i in np.unique(y):
            prob = np.mean(y == c_i)
            summation += prob**2
        return 1 - summation

    def _make_split(self, X, y, split_index, split_value):
        """Return the subsets of the dataset for the given split index & value.

        Call the method like this:
        >>> X1, y1, X2, y2 = self._make_split(X, y, split_index, split_value)

        Parameters
        ----------
        X: 2d numpy array
            Feature matrix.
        y: 1d numpy array
            Label matrix.
        split_index: int
            Index of the feature to split on.
        split_value: int/float/bool/str
            Value of feature to split on.

        Returns
        -------
        X1: 2d numpy array
            Feature matrix for subset 1
        y1: 1d numpy array
            Labels for subset 1
        X2: 2d numpy array
            Feature matrix for subset 2
        y2: 1d numpy array
            Labels for subset 2
        """
        if self.categorical[split_index]:
            idx = X[:, split_index] == split_value
        else:
            idx = X[:, split_index] < split_value
        return X[idx], y[idx], X[~idx], y[~idx]

    def _information_gain(self, y, y1, y2):
        """Return the information gain of making the given split.

        Use self.impurity_criterion(y) rather than calling _entropy or _gini
        directly.

        Parameters
        ----------
        y: 1d numpy array
            Labels for parent node.
        y1: 1d numpy array
            Labels for subset node 1.
        y2: 1d numpy array
            Labels for subset node 2.

        Returns
        -------
        float
            The information gain of making the given split.
        """
        n = y.shape[0]
        weighted_child_imp = 0
        for y_i in (y1, y2):
            weighted_child_imp += self.impurity_criterion(y_i) * y_i.shape[0] / n
        return self.impurity_criterion(y) - weighted_child_imp

    def _choose_split_index(self, X, y):
        """Return the index and value of the feature to split on.

        Determine which feature and value to split on. Return the index and
        value of the optimal split along with the split of the dataset.

        Return None, None, None if there is no split which improves information
        gain.

        Call the method like this:
        >>> index, value, splits = self._choose_split_index(X, y)
        >>> X1, y1, X2, y2 = splits

        Parameters
        ----------
            - X: 2d numpy array
            - y: 1d numpy array

        Returns
        -------
        index: int
            Index of feature
        value: int/float/bool/str
            Value of feature
        splits: tuple
            (2d array, 1d array, 2d array, 1d array)
        """
        split_index, split_value, splits = None, None, None
        best_gain = 0
        for i in range(X.shape[1]):
            values = np.unique(X[:, i])
            if len(values) <= 1:
                continue
            for value in values:
                X1, y1, X2, y2 = self._make_split(X, y, i, value)
                gain = self._information_gain(y, y1, y2)
                if gain > best_gain:
                    split_index = i
                    split_value = value
                    splits = (X1, y1, X2, y2)
                    best_gain = gain
        return split_index, split_value, splits

    def predict(self, X):
        """Return an array of predictions for the feature matrix X.

        Parameters
        ----------
        X: 2d numpy array
            The feature matrix.

        Returns
        -------
        y: 1d numpy array
            Matrix of predicted labels.
        """
        return np.array([self.root.predict_one(row) for row in X])

    def __str__(self):
        """Return string representation of the Decision Tree."""
        return str(self.root)


In [16]:
# DecisionTreePruning.py

"""
CART decision tree algorithm with pre/post pruning implemented.

TODO: Clean up multi-line lambda function (not pep8 standards)
"""

# import pandas as pd
# import numpy as np
# import math
# from collections import Counter
# from TreeNode import TreeNode


class DecisionTreePruning(object):
    """Classifier decision tree with pre and post pruning implemented.

    Parameters
    ----------
    impurity_criterion: string, optional (default='entropy')
        String indicating the impurity_criterion to use.
        Use 'gini' to have tree us Gini impurity.

    leaf_size: int, optional (default=None)
        The maxiumum number of samples in a leaf. Pre-pruning parameter.

    depth: int, optional (default=None)
        The maximum depth the tree should reach. Pre-pruning parameter.

    same_ratio: float, optional (default=None)
        The maximum ratio for instances of the same class in a leaf node.
        Pre-pruning parameter.

    error_threshold: float, optional (default=None)
        The minimum information gain threshold for a split.
        Pre-pruning parameter.
    """

    def __init__(self, impurity_criterion='entropy', leaf_size=None,
                 depth=None, same_ratio=None, error_threshold=None):
        """Initialize an empty DecisionTreePruning."""
        # Root Node
        self.root = None
        # String names of features (for interpreting the tree)
        self.feature_names = None
        # Boolean array of whether variable is categorical (or continuous)
        self.categorical = None

        if impurity_criterion == 'entropy':
            self.impurity_criterion = self._entropy
        else:
            self.impurity_criterion = self._gini

        self.leaf_size = leaf_size
        self.depth = depth
        self.same_ratio = same_ratio
        self.error_threshold = error_threshold

    def fit(self, X, y, feature_names=None):
        """Build the decision tree.

        Parameters
        ----------
        X: 2d numpy array, shape = [n_samples, n_features]
            The training data.
        y: 1d numpy array, shape = [n_samples]
            The training labels.
        feature_names: numpy array, optional (default=None)
            Array of strings containing names of each of the features.

        Returns
        -------
        None
        """
        if feature_names is None or len(feature_names) != X.shape[1]:
            self.feature_names = np.arange(X.shape[1])
        else:
            self.feature_names = feature_names

        # Create True/False array of whether the variable is categorical
        is_categorical = lambda x: isinstance(x, str) or \
                                   isinstance(x, bool) or \
                                   isinstance(x, unicode)
        self.categorical = np.vectorize(is_categorical)(X[0])

        self.root = self._build_tree(X, y)

    def _pre_prune(self, y, splits, depth):
        """Return True if any stopping threshold has been reached.

        Check pre-prunning parameters and return True/False if any stopping
        threshold has been reached.

        Parameters
        ----------
        y: 1d numpy array
            Parent node
        splits: tuple of numpy arrays
            Child nodes, tuple = (X1, y1, X2, y2)
        depth: int
            Maximum depth to build tree.

        Returns
        -------
        Boolean
        """
        X1, y1, X2, y2 = splits

        if self.leaf_size is not None:
            if self.leaf_size >= X1.shape[0] or self.leaf_size >= X2.shape[0]:
                return True
        elif self.depth is not None and depth >= self.depth:
            return True
        elif self.same_ratio is not None:
            y1_ratio = Counter(y1).most_common(1) / float(y1.shape[0])
            y2_ratio = Counter(y2).most_common(1) / float(y2.shape[0])
            if y1_ratio >= self.same_ratio or y2_ratio >= self.same_ratio:
                return True
        elif self.error_threshold is not None:
            if self.error_threshold > self._information_gain(y, y1, y2):
                return True
        return False

    def _build_tree(self, X, y, depth=-1):
        """Recursively build the decision tree.

        Return the root node.

        Parameters
        ----------
        X: 2d numpy array, shape = [n_samples, n_features]
            The training data.
        y: 1d numpy array, shape = [n_samples]

        Returns
        -------
        TreeNode
        """
        depth += 1

        node = TreeNode()
        index, value, splits = self._choose_split_index(X, y)

        if splits is not None:
            preprune = self._pre_prune(y, splits, depth)
        else:
            preprune = False

        if index is None or len(np.unique(y)) == 1 or preprune:
            node.leaf = True
            node.classes = Counter(y)
            node.name = node.classes.most_common(1)[0][0]
        else:
            X1, y1, X2, y2 = splits
            node.column = index
            node.name = self.feature_names[index]
            node.value = value
            node.categorical = self.categorical[index]
            node.left = self._build_tree(X1, y1, depth)
            node.right = self._build_tree(X2, y2, depth)
        return node

    def _entropy(self, y):
        """Return the entropy of the array y.

        Parameters
        ----------
        y: 1d numpy array
            An array of data.

        Returns
        -------
        float
            Entropy of the array y.
        """
        n = y.shape[0]
        summation = 0
        for c_i in np.unique(y):
            prob = sum(y == c_i) / float(n)
            summation += prob * np.log2(prob)
        return -summation

    def _gini(self, y):
        """Return the gini impurity of the array y.

        Parameters
        ----------
        y: 1d numpy array
            An array of data

        Returns
        -------
        float
            Gini impurity of the array y.
        """
        n = y.shape[0]
        summation = 0
        for c_i in np.unique(y):
            prob = sum(y == c_i) / float(n)
            summation += prob**2
        return 1 - summation

    def _make_split(self, X, y, split_index, split_value):
        """Return the subsets of the dataset for the given split index & value.

        Call the method like this:
        >>> X1, y1, X2, y2 = self._make_split(X, y, split_index, split_value)

        Parameters
        ----------
        X: 2d numpy array
            Feature matrix.
        y: 1d numpy array
            Label matrix.
        split_index: int
            Index of the feature to split on.
        split_value: int/float/bool/str
            Value of feature to split on.

        Returns
        -------
        X1: 2d numpy array
            Feature matrix for subset 1
        y1: 1d numpy array
            Labels for subset 1
        X2: 2d numpy array
            Feature matrix for subset 2
        y2: 1d numpy array
            Labels for subset 2
        """
        if self.categorical[split_index]:
            idx = X[:, split_index] == split_value
        else:
            idx = X[:, split_index] < split_value
        return X[idx], y[idx], X[idx == False], y[idx == False]

    def _information_gain(self, y, y1, y2):
        """Return the information gain of making the given split.

        Use self.impurity_criterion(y) rather than calling _entropy or _gini
        directly.

        Parameters
        ----------
        y: 1d numpy array
            Labels for parent node.
        y1: 1d numpy array
            Labels for subset node 1.
        y2: 1d numpy array
            Labels for subset node 2.

        Returns
        -------
        float
            The information gain of making the given split.
        """
        n = y.shape[0]
        child_inf = 0
        for y_i in (y1, y2):
            child_inf += self.impurity_criterion(y_i) * y_i.shape[0] / float(n)
        return self.impurity_criterion(y) - child_inf

    def _choose_split_index(self, X, y):
        """Return the index and value of the feature to split on.

        Determine which feature and value to split on. Return the index and
        value of the optimal split along with the split of the dataset.

        Return None, None, None if there is no split which improves information
        gain.

        Call the method like this:
        >>> index, value, splits = self._choose_split_index(X, y)
        >>> X1, y1, X2, y2 = splits

        Parameters
        ----------
            - X: 2d numpy array
            - y: 1d numpy array

        Returns
        -------
        index: int
            Index of feature
        value: int/float/bool/str
            Value of feature
        splits: tuple
            (2d array, 1d array, 2d array, 1d array)
        """
        split_index, split_value, splits = None, None, None
        gain = 0
        for i in xrange(X.shape[1]):
            values = np.unique(X[:, i])
            if len(values) < 1:
                continue
            for value in values:
                X1, y1, X2, y2 = self._make_split(X, y, i, value)
                new_gain = self._information_gain(y, y1, y2)
                if new_gain > gain:
                    split_index = i
                    split_value = value
                    splits = (X1, y1, X2, y2)
                    gain = new_gain
        return split_index, split_value, splits

    def prune(self, X, y, node=None):
        """Post-prune tree by merging leaves using error rate.

        Recursively checks for leaves and compares error rate before and after
        merging the leaves.  If merged improves error rate, merge leaves.

        Parameters
        ----------
        X: 2d numpy array, shape = [n_samples, 2]
            Feature matrix.
        y: 1d numpy array, [n_samples]
            Label matrix.
        node: TreeNode root, optional (default=None)
            The root TreeNode.

        Returns
        -------
        None
        """
        if node is None:
            node = self.root

        if not node.left.leaf:
            self.prune(X, y, node.left)

        if not node.right.leaf:
            self.prune(X, y, node.right)

        if node.left.leaf and node.right.leaf:
            leaf_y = self._predict(X, node)
            merged_classes = node.left.classes + node.right.classes
            merged_name = merged_classes.most_common(1)[0][0]
            merged_y = np.array([merged_name] * y.shape[0])
            leaf_score = sum(leaf_y == y) / float(y.shape[0])
            merged_score = sum(merged_y == y) / float(y.shape[0])

            if merged_score >= leaf_score:
                print ('Merging')
                node.leaf = True
                node.classes = merged_classes
                node.name = merged_name
                node.left = None
                node.right = None

    def _predict(self, X, node):
        """Return prediction to calculate error rate for post-pruning."""
        return np.array([node.predict_one(row) for row in X])

    def predict(self, X):
        """Return an array of predictions for the feature matrix X.

        Parameters
        ----------
        X: 2d numpy array
            The feature matrix.

        Returns
        -------
        y: 1d numpy array
            Matrix of predicted labels.
        """
        return np.array([self.root.predict_one(row) for row in X])

    def __str__(self):
        """Return string representation of the Decision Tree."""
        return str(self.root)


In [17]:
# DecisionTreeRegressor.py
"""
Regression tree algorithm with pre/post pruning implemented.

TODO: Clean up multi-line lambda function (not pep8 standards)
"""

# import pandas as pd
# import numpy as np
# import math
# from collections import Counter
# from TreeNode import TreeNode


class DecisionTreeRegressor(object):
    """Regression tree class.

    Parameters
    ----------
    leaf_size: int, optional (default=None)
        The maxiumum number of samples in a leaf. Pre-pruning parameter.

    depth: int, optional (default=None)
        The maximum depth the tree should reach. Pre-pruning parameter.

    same_ratio: float, optional (default=None)
        The maximum ratio for instances of the same class in a leaf node.
        Pre-pruning parameter.

    error_threshold: float, optional (default=None)
        The minimum information gain threshold for a split.
        Pre-pruning parameter.
    """

    def __init__(self, leaf_size=None, depth=None, same_ratio=None,
                 error_threshold=None):
        """Initialize an empty DecisionTreeRegressor."""
        # Root Node
        self.root = None
        # String names of features (for interpreting the tree)
        self.feature_names = None
        # Boolean array of whether variable is categorical (or continuous)
        self.categorical = None

        self.leaf_size = leaf_size
        self.depth = depth
        self.same_ratio = same_ratio
        self.error_threshold = error_threshold

    def fit(self, X, y, feature_names=None):
        """Build the decision tree.

        Parameters
        ----------
        X: 2d numpy array, shape = [n_samples, n_features]
            The training data.
        y: 1d numpy array, shape = [n_samples]
            The training labels.
        feature_names: numpy array, optional (default=None)
            Array of strings containing names of each of the features.

        Returns
        -------
        None
        """
        if feature_names is None or len(feature_names) != X.shape[1]:
            self.feature_names = np.arange(X.shape[1])
        else:
            self.feature_names = feature_names

        # Create True/False array of whether the variable is categorical
        is_categorical = lambda x: isinstance(x, str) or \
                                   isinstance(x, bool) or \
                                   isinstance(x, unicode)
        self.categorical = np.vectorize(is_categorical)(X[0])

        self.root = self._build_tree(X, y)

    def _pre_prune(self, y, splits, depth):
        """Return True if any stopping threshold has been reached.

        Check pre-prunning parameters and return True/False if any stopping
        threshold has been reached.

        Parameters
        ----------
        y: 1d numpy array
            Parent node
        splits: tuple of numpy arrays
            Child nodes, tuple = (X1, y1, X2, y2)
        depth: int
            Maximum depth to build tree.

        Returns
        -------
        Boolean
        """
        X1, y1, X2, y2 = splits

        if self.leaf_size is not None:
            if self.leaf_size >= X1.shape[0] or self.leaf_size >= X2.shape[0]:
                return True
        elif self.depth is not None and depth >= self.depth:
            return True
        elif self.same_ratio is not None:
            y1_ratio = Counter(y1).most_common(1) / float(y1.shape[0])
            y2_ratio = Counter(y2).most_common(1) / float(y2.shape[0])
            if y1_ratio >= self.same_ratio or y2_ratio >= self.same_ratio:
                return True
        elif self.error_threshold is not None:
            if self.error_threshold > self._information_gain(y, y1, y2):
                return True
        return False

    def _build_tree(self, X, y, depth=-1):
        """Recursively build the decision tree.

        Return the root node.

        Parameters
        ----------
        X: 2d numpy array, shape = [n_samples, n_features]
            The training data.
        y: 1d numpy array, shape = [n_samples]

        Returns
        -------
        TreeNode
        """
        depth += 1

        node = TreeNode()
        index, value, splits = self._choose_split_index(X, y)

        if splits is not None:
            preprune = self._pre_prune(y, splits, depth)
        else:
            preprune = False

        if index is None or len(np.unique(y)) == 1 or preprune:
            node.leaf = True
            node.classes = y
            node.name = y.mean()
        else:
            X1, y1, X2, y2 = splits
            node.column = index
            node.name = self.feature_names[index]
            node.value = value
            node.categorical = self.categorical[index]
            node.left = self._build_tree(X1, y1, depth)
            node.right = self._build_tree(X2, y2, depth)
        return node

    def _standard_deviaton(self, y):
        """Return the _standard_deviaton of the array.

        Parameters
        ----------
        y: 1d numpy array
            Array of targets to predict.

        Returns
        -------
        float
        """
        y_mean = y.mean()
        n = y.shape[0]
        if n == 0:
            return 0
        return np.sqrt(sum((y - y_mean)**2) / float(n))

    def _make_split(self, X, y, split_index, split_value):
        """Return the subsets of the dataset for the given split index & value.

        Call the method like this:
        >>> X1, y1, X2, y2 = self._make_split(X, y, split_index, split_value)

        Parameters
        ----------
        X: 2d numpy array
            Feature matrix.
        y: 1d numpy array
            Label matrix.
        split_index: int
            Index of the feature to split on.
        split_value: int/float/bool/str
            Value of feature to split on.

        Returns
        -------
        X1: 2d numpy array
            Feature matrix for subset 1
        y1: 1d numpy array
            Labels for subset 1
        X2: 2d numpy array
            Feature matrix for subset 2
        y2: 1d numpy array
            Labels for subset 2
        """
        if self.categorical[split_index]:
            idx = X[:, split_index] == split_value
        else:
            idx = X[:, split_index] < split_value
        return X[idx], y[idx], X[idx == False], y[idx == False]

    def _std_reduction(self, y, y1, y2):
        """Return the standard deviation reduction of making the given split.

        Parameters
        ----------
        y: 1d numpy array
            Targets for parent node.
        y1: 1d numpy array
            Targets for subset 1.
        y2: 1d numpy array
            Targets for subset 2.

        Returns
        -------
        float
        """
        n = y.shape[0]
        child_std = 0
        for y_i in (y1, y2):
            child_std += self._standard_deviaton(y_i) * y_i.shape[0] / float(n)
        return self._standard_deviaton(y) - child_std

    def _choose_split_index(self, X, y):
        """Return the index and value of the feature to split on.

        Determine which feature and value to split on. Return the index and
        value of the optimal split along with the split of the dataset.

        Return None, None, None if there is no split which improves information
        gain.

        Call the method like this:
        >>> index, value, splits = self._choose_split_index(X, y)
        >>> X1, y1, X2, y2 = splits

        Parameters
        ----------
            - X: 2d numpy array
            - y: 1d numpy array

        Returns
        -------
        index: int
            Index of feature
        value: int/float/bool/str
            Value of feature
        splits: tuple
            (2d array, 1d array, 2d array, 1d array)
        """
        split_index, split_value, splits = None, None, None
        std_reduction = 0
        for i in xrange(X.shape[1]):
            values = np.unique(X[:, i])
            if len(values) < 1:
                continue
            for value in values:
                X1, y1, X2, y2 = self._make_split(X, y, i, value)
                new_std_reduction = self._std_reduction(y, y1, y2)
                if new_std_reduction > std_reduction:
                    split_index = i
                    split_value = value
                    splits = (X1, y1, X2, y2)
                    std_reduction = new_std_reduction
        return split_index, split_value, splits

    def prune(self, X, y, node=None):
        """Post-prune tree by merging leaves using error rate.

        Recursively checks for leaves and compares error rate before and after
        merging the leaves.  If merged improves error rate, merge leaves.

        Parameters
        ----------
        X: 2d numpy array, shape = [n_samples, 2]
            Feature matrix.
        y: 1d numpy array, [n_samples]
            Label matrix.
        node: TreeNode root, optional (default=None)
            The root TreeNode.

        Returns
        -------
        None
        """
        if node is None:
            node = self.root

        if not node.left.leaf:
            self.prune(X, y, node.left)

        if not node.right.leaf:
            self.prune(X, y, node.right)

        if node.left.leaf and node.right.leaf:
            leaf_y = self.predict(X)
            merged_classes = np.concatenate((node.left.classes,
                                             node.right.classes))
            merged_name = merged_classes.mean()
            merged_y = merged_name
            leaf_score = sum((y - leaf_y)**2)
            merged_score = sum((y - merged_y)**2)

            if merged_score <= leaf_score:
                node.leaf = True
                node.classes = merged_classes
                node.name = merged_name
                node.left = None
                node.right = None

    def predict(self, X):
        """Return an array of predictions for the feature matrix X.

        Parameters
        ----------
        X: 2d numpy array
            The feature matrix.

        Returns
        -------
        y: 1d numpy array
            Matrix of predicted labels.
        """
        return np.apply_along_axis(self.root.predict_one, axis=1, arr=X)

    def __str__(self):
        """Return string representation of the Decision Tree."""
        return str(self.root)


In [11]:
# run_decision_tree.py
"""
Class implementing the CART decision tree algorithm.

"""

# import pandas as pd
# import numpy as np
# import math
# from collections import Counter
# from TreeNode import TreeNode


class DecisionTree(object):
    """Classifier implementing the CART decision tree algorithm.

    Parameters
    ----------
    impurity_criterion: string, optional (default='entropy')
        String indicating the impurity_criterion to use.
        Use 'gini' to have tree us Gini impurity.
    """

    def __init__(self, impurity_criterion='entropy'):
        """Initialize an empty DecisionTree."""
        # Root Node
        self.root = None
        # String names of features (for interpreting the tree)
        self.feature_names = None
        # Boolean array of whether variable is categorical (or continuous)
        self.categorical = None

        if impurity_criterion == 'entropy':
            self.impurity_criterion = self._entropy
        else:
            self.impurity_criterion = self._gini


    def fit(self, X, y, feature_names=None):
        """Build the decision tree.

        Parameters
        ----------
        X: 2d numpy array, shape = [n_samples, n_features]
            The training data.
        y: 1d numpy array, shape = [n_samples]
            The training labels.
        feature_names: numpy array, optional (default=None)
            Array of strings containing names of each of the features.

        Returns
        -------
        None
        """
        if feature_names is None or len(feature_names) != X.shape[1]:
            self.feature_names = np.arange(X.shape[1])
        else:
            self.feature_names = feature_names

        # Create True/False array of whether the variable is categorical
        is_categorical = lambda x: isinstance(x, str) or isinstance(x, bool)
        self.categorical = np.vectorize(is_categorical)(X[0])

        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y):
        """Recursively build the decision tree.

        Return the root node.

        Parameters
        ----------
        X: 2d numpy array, shape = [n_samples, n_features]
            The training data.
        y: 1d numpy array, shape = [n_samples]

        Returns
        -------
        TreeNode
        """
        node = TreeNode()
        index, value, splits = self._choose_split_index(X, y)

        if index is None or len(np.unique(y)) == 1:
            node.leaf = True
            node.classes = Counter(y)
            node.name = node.classes.most_common(1)[0][0]
        else:
            X1, y1, X2, y2 = splits
            node.column = index
            node.name = self.feature_names[index]
            node.value = value
            node.categorical = self.categorical[index]
            node.left = self._build_tree(X1, y1)
            node.right = self._build_tree(X2, y2)
        return node

    def _entropy(self, y):
        """Return the entropy of the array y.

        Parameters
        ----------
        y: 1d numpy array
            An array of data.

        Returns
        -------
        float
            Entropy of the array y.
        """
        n = y.shape[0]
        summation = 0
        for c_i in np.unique(y):
            prob = np.mean(y == c_i)
            summation += prob * np.log2(prob)
        return -summation

    def _gini(self, y):
        """Return the gini impurity of the array y.

        Parameters
        ----------
        y: 1d numpy array
            An array of data

        Returns
        -------
        float
            Gini impurity of the array y.
        """
        n = y.shape[0]
        summation = 0
        for c_i in np.unique(y):
            prob = np.mean(y == c_i)
            summation += prob**2
        return 1 - summation

    def _make_split(self, X, y, split_index, split_value):
        """Return the subsets of the dataset for the given split index & value.

        Call the method like this:
        >>> X1, y1, X2, y2 = self._make_split(X, y, split_index, split_value)

        Parameters
        ----------
        X: 2d numpy array
            Feature matrix.
        y: 1d numpy array
            Label matrix.
        split_index: int
            Index of the feature to split on.
        split_value: int/float/bool/str
            Value of feature to split on.

        Returns
        -------
        X1: 2d numpy array
            Feature matrix for subset 1
        y1: 1d numpy array
            Labels for subset 1
        X2: 2d numpy array
            Feature matrix for subset 2
        y2: 1d numpy array
            Labels for subset 2
        """
        if self.categorical[split_index]:
            idx = X[:, split_index] == split_value
        else:
            idx = X[:, split_index] < split_value
        return X[idx], y[idx], X[~idx], y[~idx]

    def _information_gain(self, y, y1, y2):
        """Return the information gain of making the given split.

        Use self.impurity_criterion(y) rather than calling _entropy or _gini
        directly.

        Parameters
        ----------
        y: 1d numpy array
            Labels for parent node.
        y1: 1d numpy array
            Labels for subset node 1.
        y2: 1d numpy array
            Labels for subset node 2.

        Returns
        -------
        float
            The information gain of making the given split.
        """
        n = y.shape[0]
        weighted_child_imp = 0
        for y_i in (y1, y2):
            weighted_child_imp += self.impurity_criterion(y_i) * y_i.shape[0] / n
        return self.impurity_criterion(y) - weighted_child_imp

    def _choose_split_index(self, X, y):
        """Return the index and value of the feature to split on.

        Determine which feature and value to split on. Return the index and
        value of the optimal split along with the split of the dataset.

        Return None, None, None if there is no split which improves information
        gain.

        Call the method like this:
        >>> index, value, splits = self._choose_split_index(X, y)
        >>> X1, y1, X2, y2 = splits

        Parameters
        ----------
            - X: 2d numpy array
            - y: 1d numpy array

        Returns
        -------
        index: int
            Index of feature
        value: int/float/bool/str
            Value of feature
        splits: tuple
            (2d array, 1d array, 2d array, 1d array)
        """
        split_index, split_value, splits = None, None, None
        best_gain = 0
        for i in range(X.shape[1]):
            values = np.unique(X[:, i])
            if len(values) <= 1:
                continue
            for value in values:
                X1, y1, X2, y2 = self._make_split(X, y, i, value)
                gain = self._information_gain(y, y1, y2)
                if gain > best_gain:
                    split_index = i
                    split_value = value
                    splits = (X1, y1, X2, y2)
                    best_gain = gain
        return split_index, split_value, splits

    def predict(self, X):
        """Return an array of predictions for the feature matrix X.

        Parameters
        ----------
        X: 2d numpy array
            The feature matrix.

        Returns
        -------
        y: 1d numpy array
            Matrix of predicted labels.
        """
        return np.array([self.root.predict_one(row) for row in X])

    def __str__(self):
        """Return string representation of the Decision Tree."""
        return str(self.root)


In [18]:
# run_decision_tree_pruning.py
# import pandas as pd
# from itertools import izip
# from DecisionTreePruning import DecisionTreePruning


def test_tree(filename):
    df = pd.read_csv(filename)
    y = df.pop('Result').values
    X = df.values
    print(X)
    
    #tree = DecisionTree(leaf_size=10, depth=5, same_ratio=0.8, error_threshold=0.1)
    tree = DecisionTreePruning()
    tree.fit(X, y, df.columns)
    tree.prune(X, y)
    print(tree)

    y_predict = tree.predict(X)
    print '%26s   %10s   %10s' % ("FEATURES", "ACTUAL", "PREDICTED")
    print '%26s   %10s   %10s' % ("----------", "----------", "----------")
    for features, true, predicted in izip(X, y, y_predict):
        print '%26s   %10s   %10s' % (str(features), str(true), str(predicted))


if __name__ == '__main__':
    test_tree('data/playgolf.csv')

SyntaxError: invalid syntax (<ipython-input-18-13394491bb0d>, line 20)

In [10]:
# run_decision_tree_regressor.py
# import pandas as pd
# from itertools import izip
# from DecisionTreeRegressor import DecisionTreeRegressor
#!/usr/bin/env python2.7

def test_tree(filename):
    df = pd.read_csv(filename)
    y = df.pop('Humidity').values
    X = df.values
    print(X)
    
    tree = DecisionTreeRegressor()
    tree.fit(X, y, df.columns)
    tree.prune(X, y)
    print(tree)

    y_predict = tree.predict(X)
    print '%35s   %10s   %10s' % ("FEATURES", "ACTUAL", "PREDICTED")
    print '%35s   %10s   %10s' % ("----------", "----------", "----------")
    for features, true, predicted in izip(X, y, y_predict):
        print '%35s   %10d   %10d' % (str(features), true, predicted)


if __name__ == '__main__':
    test_tree('data/playgolf.csv')


SyntaxError: invalid syntax (<ipython-input-10-00e1c2506325>, line 19)